# Load data

In [ ]:
filename_train = '../input/tabular-playground-series-mar-2021/train.csv'
filename_test = '../input/tabular-playground-series-mar-2021/test.csv'

In [ ]:
import pandas as pd

df_train = pd.read_csv(filename_train, index_col='id')
df_train

In [ ]:
df_test = pd.read_csv(filename_test, index_col='id')
df_test

# Get explanatory and objective variables

In [ ]:
X_train = df_train.drop('target', axis=1)
X_train

In [ ]:
y_train = df_train['target']
y_train

In [ ]:
X_test = df_test.copy()
X_test

# Change object-type to category-type

In [ ]:
def to_category(df):
    for col, type in zip(df.columns, df.dtypes):
        if type == object:
            df[col] = df[col].astype('category')
    
    return df

In [ ]:
X_train = to_category(X_train)
X_test  = to_category(X_test)

# Modeling

In [ ]:
import numpy as np
import lightgbm as lgb
from sklearn.metrics import accuracy_score

def acc_score(preds, data):
    y_true = data.get_label()
    y_pred = np.where(preds > 0.5, 1, 0)
    acc = accuracy_score(y_true, y_pred)
    return 'acc', acc, True

lgb_train = lgb.Dataset(X_train, y_train)

params = {
    'objective': 'binary', 
    'metric': 'auc',
    'force_row_wise': True,
}

cv_rslt = lgb.cv(params, 
                 lgb_train, 
                 nfold=5, 
                 num_boost_round=10000, 
                 early_stopping_rounds=20,
                 metrics='auc',
                 return_cvbooster=True)

In [ ]:
cv_rslt['auc-mean'][-1]

# Prediction

In [ ]:
import scipy

cvbooster = cv_rslt['cvbooster']
y_pred = np.mean(cvbooster.predict(X_test, num_iteration=cvbooster.best_iteration), axis=0)
y_pred

In [ ]:
answer = pd.concat([df_test.index.to_series().reset_index(drop=True), 
                    pd.Series(y_pred, name='target')], axis=1)
answer

In [ ]:
filename_output = './submission.csv'
answer.to_csv(filename_output, index=False)